In [222]:
from pyignite import Client
import pandas as pd

# LASSO manager node (IP, DNS)
lasso_host = '127.0.0.1'
lasso_port = 10800

# Ignite thin client
client = Client()
client.connect(lasso_host, lasso_port)

In [223]:
# LSL script execution ID
my_script_id = '9607a33a-d59c-4109-b9e1-72847e95ef0e'

# use SQL (see https://apache-ignite-binary-protocol-client.readthedocs.io/en/latest/index.html)
cursor = client.sql("SELECT * FROM srm.CELLVALUE where executionId = ? and type != 'seq' and type != 'exseq'", query_args= [my_script_id], include_field_names=True)
column_headers = next(cursor)
rows = list(cursor)

# SQL cursor to pandas DataFrame
df = pd.DataFrame(rows, columns=column_headers)

display(df.head(5))
# print tabulated

,EXECUTIONID,ABSTRACTIONID,ACTIONID,ARENAID,SHEETID,SYSTEMID,VARIANTID,ADAPTERID,X,Y,TYPE,VALUE,RAWVALUE,VALUETYPE,LASTMODIFIED,EXECUTIONTIME
0,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,jacoco,all,fead0d3f-6502-45e8-bde6-52eb34bfd76f,original,74,-1,-1,jacoco_class_branch.total,4,None,None,"(2024-08-31 10:33:22.632000, 0)",0
1,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_0c9ae82c-1aa5-4392-9920-83128053dd07,0c9ae82c-1aa5-4392-9920-83128053dd07,original,87,2,5,input_value,"{""elements"":[""hello"",""world"",null,null,null,nu...","{""elements"":[""hello"",""world"",null,null,null,nu...",edu.columbia.cs.psl.phosphor.struct.ArrayList,"(2024-08-31 10:30:31.083000, 0)",379542
2,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,jacoco,all,444493ee-e374-4ba0-8a9f-9cb9cc770eab,original,54,-1,-1,jacoco_class_line.missed,24,None,None,"(2024-08-31 10:31:37.294000, 0)",0
3,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,jacoco,pushPop_fead0d3f-6502-45e8-bde6-52eb34bfd76f,fead0d3f-6502-45e8-bde6-52eb34bfd76f,original,68,0,9,value,_NOT_EXECUTED_,None,None,"(2024-08-31 10:33:20.357000, 0)",-1
4,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,jacoco,all,fead0d3f-6502-45e8-bde6-52eb34bfd76f,original,13,-1,-1,jacoco_class_method.covered,4,None,None,"(2024-08-31 10:32:30.199000, 0)",0


In [224]:
# IMPLEMENTATION ID (CODE UNIT + ADAPTER) 
df['IMPLID'] = df['SYSTEMID'] + '_' + df['ADAPTERID']
# STATEMENT ID (SHEETS AND COORDINATES)
df['STATEMENT'] = df['SHEETID'].str.rsplit(pat='_', n=1, expand=True)[0] + '@' + df['X'].astype(str) + ',' + df['Y'].astype(str)

In [225]:
display(df["ADAPTERID"].nunique())

base64_outputs = df.query('TYPE == "value" & ARENAID == "execute" & SHEETID != "all"').sort_values(by=['STATEMENT'])
base64_outputs.head(5)

101

,EXECUTIONID,ABSTRACTIONID,ACTIONID,ARENAID,SHEETID,SYSTEMID,VARIANTID,ADAPTERID,X,Y,TYPE,VALUE,RAWVALUE,VALUETYPE,LASTMODIFIED,EXECUTIONTIME,IMPLID,STATEMENT
46204,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_b9650f0f-b409-41b7-99b3-effb5ae63fd7,b9650f0f-b409-41b7-99b3-effb5ae63fd7,original,2,0,0,value,_INSTANCE_,edu.columbia.cs.psl.phosphor.struct.SinglyLink...,edu.columbia.cs.psl.phosphor.struct.SinglyLink...,"(2024-08-31 10:32:24.187000, 0)",602583,b9650f0f-b409-41b7-99b3-effb5ae63fd7_2,"pushPop@0,0"
57558,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_0c9ae82c-1aa5-4392-9920-83128053dd07,0c9ae82c-1aa5-4392-9920-83128053dd07,original,62,0,0,value,_INSTANCE_,edu.columbia.cs.psl.phosphor.struct.ArrayList,edu.columbia.cs.psl.phosphor.struct.ArrayList,"(2024-08-31 10:30:30.037000, 0)",720500,0c9ae82c-1aa5-4392-9920-83128053dd07_62,"pushPop@0,0"
27181,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_e67884d4-a4f1-4e5e-97be-18914804b74b,e67884d4-a4f1-4e5e-97be-18914804b74b,original,66,0,0,value,_INSTANCE_,edu.columbia.cs.psl.phosphor.struct.LinkedList,edu.columbia.cs.psl.phosphor.struct.LinkedList,"(2024-08-31 10:27:38.779000, 0)",817500,e67884d4-a4f1-4e5e-97be-18914804b74b_66,"pushPop@0,0"
27132,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_fead0d3f-6502-45e8-bde6-52eb34bfd76f,fead0d3f-6502-45e8-bde6-52eb34bfd76f,original,3,0,0,value,_INSTANCE_,toools.collections.ArrayFIFO,toools.collections.ArrayFIFO,"(2024-08-31 10:32:11.748000, 0)",779625,fead0d3f-6502-45e8-bde6-52eb34bfd76f_3,"pushPop@0,0"
57618,9607a33a-d59c-4109-b9e1-72847e95ef0e,Stack,filter,execute,pushPop_444493ee-e374-4ba0-8a9f-9cb9cc770eab,444493ee-e374-4ba0-8a9f-9cb9cc770eab,original,20,0,0,value,_INSTANCE_,edu.columbia.cs.psl.phosphor.struct.WeakIdenti...,edu.columbia.cs.psl.phosphor.struct.WeakIdenti...,"(2024-08-31 10:30:31.234000, 0)",1535417,444493ee-e374-4ba0-8a9f-9cb9cc770eab_20,"pushPop@0,0"


In [226]:
base64_outputs.drop_duplicates(subset=['IMPLID', 'STATEMENT', 'VALUE'], inplace=True)

# pivot the data to allow column-wise comparisons
pivot_table = base64_outputs.pivot(index='STATEMENT', columns= 'IMPLID', values= 'VALUE').sort_values(by=['STATEMENT'])

display(pivot_table)

IMPLID,0c9ae82c-1aa5-4392-9920-83128053dd07_0,0c9ae82c-1aa5-4392-9920-83128053dd07_1,0c9ae82c-1aa5-4392-9920-83128053dd07_10,0c9ae82c-1aa5-4392-9920-83128053dd07_11,0c9ae82c-1aa5-4392-9920-83128053dd07_12,0c9ae82c-1aa5-4392-9920-83128053dd07_13,0c9ae82c-1aa5-4392-9920-83128053dd07_14,0c9ae82c-1aa5-4392-9920-83128053dd07_15,0c9ae82c-1aa5-4392-9920-83128053dd07_16,0c9ae82c-1aa5-4392-9920-83128053dd07_17,...,fead0d3f-6502-45e8-bde6-52eb34bfd76f_84,fead0d3f-6502-45e8-bde6-52eb34bfd76f_86,fead0d3f-6502-45e8-bde6-52eb34bfd76f_89,fead0d3f-6502-45e8-bde6-52eb34bfd76f_9,fead0d3f-6502-45e8-bde6-52eb34bfd76f_91,fead0d3f-6502-45e8-bde6-52eb34bfd76f_94,fead0d3f-6502-45e8-bde6-52eb34bfd76f_95,fead0d3f-6502-45e8-bde6-52eb34bfd76f_96,fead0d3f-6502-45e8-bde6-52eb34bfd76f_97,oracle_oracle
STATEMENT,,,,,,,,,,,,,,,,,,,,,
"pushPop@0,0",_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,...,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_,_INSTANCE_
"pushPop@0,1",null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,"""hello"""
"pushPop@0,2",null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,"""world"""
"pushPop@0,3",null,_EXCEPTION_,null,_EXCEPTION_,_EXCEPTION_,_EXCEPTION_,_EXCEPTION_,null,_EXCEPTION_,_EXCEPTION_,...,"""hello""","""hello""","""hello""","""hello""","""hello""","""hello""","""hello""","""hello""","""hello""","""world"""
"pushPop@0,4",2,_NOT_EXECUTED_,2,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,2,_NOT_EXECUTED_,_NOT_EXECUTED_,...,_EXCEPTION_,_EXCEPTION_,_EXCEPTION_,2,_EXCEPTION_,_EXCEPTION_,1,2,1,2
"pushPop@0,5",false,_NOT_EXECUTED_,false,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_EXCEPTION_,_NOT_EXECUTED_,_NOT_EXECUTED_,...,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_EXCEPTION_,_NOT_EXECUTED_,_NOT_EXECUTED_,_EXCEPTION_,null,null,false
"pushPop@0,6",_EXCEPTION_,_NOT_EXECUTED_,_EXCEPTION_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,...,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,"""hello""","""world""","""world"""
"pushPop@0,7",_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,...,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,1,1,1
"pushPop@0,8",_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,...,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,_NOT_EXECUTED_,"""world""","""world""","""hello"""


In [241]:
pivot_transpose = pivot_table.transpose()
pivot_transpose = pivot_transpose.reset_index()
correct_row = pivot_transpose[pivot_transpose["IMPLID"].str.startswith('oracle')].reset_index()
pivot_transpose['artifactid'] = pivot_transpose["IMPLID"].apply(lambda x: x[:-2])
# print(pivot_transpose[
#     (pivot_transpose["pushPop@0,2"] == correct_row.loc[0,"pushPop@0,2"]) 
#     & (pivot_transpose["pushPop@0,3"] == correct_row.loc[0,"pushPop@0,3"])
#     & (pivot_transpose["pushPop@0,4"] == correct_row.loc[0,"pushPop@0,4"]) 
#     & (pivot_transpose["pushPop@0,5"] == correct_row.loc[0,"pushPop@0,5"])
#     & (pivot_transpose["pushPop@0,6"] == correct_row.loc[0,"pushPop@0,6"]) 
#     & (pivot_transpose["pushPop@0,7"] == correct_row.loc[0,"pushPop@0,7"])
#     & (pivot_transpose["pushPop@0,8"] == correct_row.loc[0,"pushPop@0,8"]) 
#     & (pivot_transpose["pushPop@0,9"] == correct_row.loc[0,"pushPop@0,9"])
#     ]
#     ["artifactid"].unique())
# print(pivot_transpose[
#     (pivot_transpose["pushPop@0,2"] != correct_row.loc[0,"pushPop@0,2"]) 
#     | (pivot_transpose["pushPop@0,3"] != correct_row.loc[0,"pushPop@0,3"])
#     | (pivot_transpose["pushPop@0,4"] != correct_row.loc[0,"pushPop@0,4"]) 
#     | (pivot_transpose["pushPop@0,5"] != correct_row.loc[0,"pushPop@0,5"])
#     | (pivot_transpose["pushPop@0,6"] != correct_row.loc[0,"pushPop@0,6"]) 
#     | (pivot_transpose["pushPop@0,7"] != correct_row.loc[0,"pushPop@0,7"])
#     | (pivot_transpose["pushPop@0,8"] != correct_row.loc[0,"pushPop@0,8"]) 
#     | (pivot_transpose["pushPop@0,9"] != correct_row.loc[0,"pushPop@0,9"])
#     ]
#     ["artifactid"].unique())
print(pivot_transpose[(pivot_transpose["pushPop@0,1"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,2"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,3"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,4"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,5"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,6"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,7"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,8"].str.startswith("_EXCEPTION")) 
    | (pivot_transpose["pushPop@0,9"].str.startswith("_EXCEPTION"))
    ]
    ["artifactid"].apply(lambda x: x.replace("_", "")).unique())


['0c9ae82c-1aa5-4392-9920-83128053dd07'
 '330bfb4b-2a08-46a6-ac2b-a878b161ef56'
 '38f0ac91-dff9-4861-a693-c865f1ab80b2'
 '444493ee-e374-4ba0-8a9f-9cb9cc770eab'
 '4cea25a5-b3c6-4d4e-8b2b-2787a55b46f4'
 '58d66e48-8083-4c53-947b-f97ac2bd3ab7'
 '5b9586b8-12b8-44f3-8d71-39118c755c4c'
 'b9650f0f-b409-41b7-99b3-effb5ae63fd7'
 'be1298a7-ad39-4a9f-8b71-8e3529b193c7'
 'e67884d4-a4f1-4e5e-97be-18914804b74b'
 'e880b28a-4ce0-4bef-9aad-a581d9523aff'
 'fead0d3f-6502-45e8-bde6-52eb34bfd76f']


In [232]:
print(len(pivot_table.columns))

print(base64_outputs["IMPLID"].nunique())
print(len(list(set(map(lambda x: x[:-2].replace("_", ""), base64_outputs["IMPLID"].unique())))))

730
730
13


In [ ]:
base64_cc_branch = df.query('TYPE == "jacoco_class_branch.covered" & ARENAID == "jacoco"')
base64_cc_branch
base64_cc_branch['VALUE'].astype(int).describe()

count    29.000000
mean      1.724138
std       5.167872
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max      17.000000
Name: VALUE, dtype: float64